In [1]:
import numpy as np
import pandas as pd
import mplfinance as mpf
import os
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import talib as ta # conda activate cryt310
import pickle 
from scipy import stats
import requests
from tqdm import tqdm
from time import sleep, perf_counter
from threading import Thread
from aver6_funcs import get_data

In [2]:
with open("trading_pairs_with_highvol_USDTonly.pkl","rb") as f:
    valid_symbols1 = pickle.load(f)
valid_symbols_ = valid_symbols1+["DF","FOR","GNO","MLN","POLS","QUICK","SUPER","GLM",
                                 "TUSD","PROM","USDC","WNXM","WLD","BIFI","FDUSD","CYBER","SEI"]
valid_symbols = []
for v in valid_symbols_:
    if v == "YFII":
        continue
    if v not in valid_symbols:
        valid_symbols.append(v)

# javascript code to retrieve the element ID

```javascript
// Your XPath expression
const xpathExpression = '/html/body/div[1]/div[4]/div/div[8]/div/div[2]/div[1]/div/div/div[6]';

// Evaluate the XPath against the document
const iterator = document.evaluate(xpathExpression, document, null, XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);

// Get the first matching node
const element = iterator.iterateNext();

// Do something with the element (e.g., change its text content)
if (element) {
    element.textContent = element.textContent.substr(0,5)+" "+element.style.top;
}
```

In [3]:
len(valid_symbols),"BUSD" in valid_symbols,"WLD" in valid_symbols

(353, True, True)

In [4]:
for ind,sym in enumerate(sorted(valid_symbols,key=lambda x : f"{x:0<6}")):
    print(f"{sym:<7}",f"{ind*24:<7}",f"{ind:<7}")

1INCH   0       0      
AAVE    24      1      
ACA     48      2      
ACH     72      3      
ACM     96      4      
ADA     120     5      
ADX     144     6      
AERGO   168     7      
AGIX    192     8      
AGLD    216     9      
AKRO    240     10     
ALCX    264     11     
ALGO    288     12     
ALICE   312     13     
ALPACA  336     14     
ALPHA   360     15     
ALPINE  384     16     
AMB     408     17     
AMP     432     18     
ANKR    456     19     
ANT     480     20     
APE     504     21     
API3    528     22     
APT     552     23     
AR      576     24     
ARB     600     25     
ARDR    624     26     
ARKM    648     27     
ARPA    672     28     
ASR     696     29     
AST     720     30     
ASTR    744     31     
ATA     768     32     
ATM     792     33     
ATOM    816     34     
AUCTION 840     35     
AUDIO   864     36     
AVA     888     37     
AVAX    912     38     
AXS     936     39     
BADGER  960     40     
BAKE    984     

# getting a list for jessie ai

In [9]:
counts = 0
for i,sym in enumerate(valid_symbols):
    if counts >11: break
    if "USD" in sym: continue
    print(sym.upper()+"-USDT")
    counts+=1

BTC-USDT
XRP-USDT
ETH-USDT
XLM-USDT
SOL-USDT
DOGE-USDT
BNB-USDT
LINK-USDT
ARB-USDT
ADA-USDT
MATIC-USDT
LTC-USDT


In [25]:
dfmpl =get_data("XEM"+"USDT","1h")

In [26]:
dfmpl

,Open,High,Low,Close,Volume
Date,,,,,
2023-08-01 22:00:00,0.03024,0.03036,0.03020,0.03034,631352.0
2023-08-01 23:00:00,0.03034,0.03057,0.03029,0.03053,1826147.0
2023-08-02 00:00:00,0.03051,0.03080,0.03039,0.03058,1355261.0
2023-08-02 01:00:00,0.03058,0.03070,0.03053,0.03061,724898.0
2023-08-02 02:00:00,0.03062,0.03104,0.03050,0.03098,1439468.0
...,...,...,...,...,...
2023-09-12 09:00:00,0.02410,0.02418,0.02401,0.02401,317186.0
2023-09-12 10:00:00,0.02401,0.02403,0.02394,0.02397,367390.0
2023-09-12 11:00:00,0.02395,0.02400,0.02385,0.02385,156009.0


In [27]:
dfmpl =get_data(valid_symbols[4]+"USDT","1h")

In [28]:
%%time
data_fetched = [[] for v in valid_symbols]
def task(id):
    #print(f'{id}{valid_symbols[id]}',end="")
    sleep(id*0.02)# 0.05 works
    dfmpl =get_data(valid_symbols[id]+"USDT","1h")
    data_fetched[id]=dfmpl.copy()
    #print(f'{id}{valid_symbols[id]}fin',end="")
    if id%10==0:
        print(f'{id}',end=",")
start_time = perf_counter()
# create and start 10 threads
threads = []
for n in range(len(valid_symbols)):
    t = Thread(target=task, args=(n,))
    threads.append(t)
    t.start()
# wait for the threads to complete
for t in threads:
    t.join()
end_time = perf_counter()
print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

0,10,30,20,40,60,50,70,80,90,110,100,120,130,160,150170,,140,200230,,180,240,250,190,340,330,290,210,280,260,220,300,270,310,320,350,It took  11.52 second(s) to complete.
CPU times: total: 13.8 s
Wall time: 11.5 s


In [29]:
%%time
data=[]
for valid_symbol_i in tqdm(range(len(valid_symbols))):
    #dfmpl =get_data(valid_symbol+"USDT","1h")
    dfmpl=data_fetched[valid_symbol_i]
    data.append((dfmpl.Open*0.5+dfmpl.Close*0.5).values)

100%|██████████████████████████████████████████████████████████████████████████████| 353/353 [00:00<00:00, 1458.44it/s]

CPU times: total: 156 ms
Wall time: 274 ms


In [30]:
from collections import Counter
Counter([len(d) for d in data])

Counter({1000: 351, 671: 2})

In [31]:
subset_symbols=[]
for i,(valid_symbol,d) in enumerate(zip(valid_symbols,data)):
    if len(d)>24:
        subset_symbols.append(valid_symbol)

In [32]:
len(subset_symbols),"WLD" in subset_symbols,"BUSD" in subset_symbols

(353, True, True)

In [33]:
# with open("9_0_subset_symbols_24hrchange.pkl","wb") as f:
#     pickle.dump(subset_symbols,f)
with open("2_1_0_subset_symbols.pkl","wb") as f:
    pickle.dump(subset_symbols,f)

# end of file

In [34]:
non_tradable = ["BUSD","USDC","TUSD","USDP","FDUSD"]
for nt in non_tradable:
    print( nt in subset_symbols,nt)

True BUSD
True USDC
True TUSD
True USDP
True FDUSD


In [35]:
for sym in subset_symbols:
    if ("USD" in sym):
        print(sym)

BUSD
USDP
TUSD
USDC
FDUSD
